In [22]:
import numpy as np
import pandas as pd

In [23]:
# Read the labels from the labels.tok file
with open('labels.tok', 'r') as labels_file:
    labels = labels_file.read().splitlines()
labels = labels[1:] # Remove the first line

In [24]:
def read_hyp_file(path):
    data = []
    with open(path, 'r') as file:
        lines = file.readlines()
        sublist = []
        for line in lines:
            line = line.strip()
            
            if line.startswith('['):
                sublist = [float(value) for value in line[1:-1].split()]
            elif line.endswith(']'):
                sublist += [float(value) for value in line[:-1].split()]
                data.append(sublist)
            else:
                sublist += [float(value) for value in line.split()]
    return data

In [25]:
# Read the file
data = read_hyp_file('hyps.txt')

In [26]:
len(data)

75492

In [27]:
import csv

def count_lines(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        line_count = sum(1 for row in reader)
    return line_count

csv_file = 'test.csv'
num_lines = count_lines(csv_file)
print(f"The CSV file contains {num_lines} lines.")

The CSV file contains 75493 lines.


In [28]:
df_predictions = pd.DataFrame(data, columns=labels)

In [29]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df_predictions.values)
df_predictions = pd.DataFrame(normalized_data, columns=df_predictions.columns)

Thresholding

In [30]:
# # create ground truth and predictions dataframes for validation set

# df_validation_pred = pd.DataFrame(read_hyp_file('validation_hyps.txt'), columns=labels)
# df_validation_gt = pd.DataFrame(read_hyp_file('validation_refs.txt'), columns=labels)

# scaler = MinMaxScaler()
# normalized_data = scaler.fit_transform(df_validation_pred.values)
# df_validation_pred = pd.DataFrame(normalized_data, columns=df_validation_pred.columns)

In [31]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ground_truth = df_validation_gt['No positive label'].values

# best_threshold = None
# best_metric_score = 0.0

# for threshold in np.linspace(0, 1, num=100):
#     predicted = np.where(df_validation_pred['No positive label'] >= threshold, 1, 0)
#     metric_score = accuracy_score(ground_truth, predicted)
#     if metric_score > best_metric_score:
#         best_threshold = threshold
#         best_metric_score = metric_score

# print("Best Threshold:", best_threshold)
# print("Best Metric Score:", best_metric_score)

In [32]:
# threshold = best_threshold
threshold = 0.98989898989899

In [33]:
# Remove all predictions that are seen as without label
df_predictions.loc[df_predictions['No positive label'] > threshold, :] = 0
df_predictions = df_predictions.drop('No positive label', axis=1)

In [34]:
csv_file_path = 'test.csv'
df_csv = pd.read_csv(csv_file_path)
first_column = df_csv.iloc[:, 0]

In [35]:
# Add dicom ids
# numbers = []
# with open('test_dicoms.tok', 'r') as file:
#     lines = file.readlines()
#     for line in lines:
#         number = line.split('/')[-1].split('.')[0]
#         numbers.append(number)

df_predictions.insert(0, 'dicom_id', first_column)

In [36]:
for column in df_predictions.columns[1:]:
    df_predictions.loc[df_predictions[column] >= 1, column] = 1
    df_predictions.loc[df_predictions[column] <= 0, column] = 0

In [37]:
df_predictions.head()

,dicom_id,Pneumomediastinum,Pneumothorax,Tortuous Aorta,Atelectasis,Pleural Other,Pleural Thickening,Fracture,Lung Opacity,Edema,...,Infiltration,Pneumonia,Calcification of the Aorta,Hernia,Nodule,Pleural Effusion,Enlarged Cardiomediastinum,Mass,Support Devices,Subcutaneous Emphysema
0,d0b71acc-b5a62046-bbb5f6b8-7b173b85-65cdf738,0.169517,0.670197,0.638955,0.144436,0.634753,0.053983,0.674476,0.642324,0.061954,...,0.276915,0.180555,0.323324,0.620464,0.291272,0.071654,0.205115,0.348954,0.203908,0.546325
1,3be619d1-506a66cf-ff1ab8a1-2efb77bb-fe7d59fc,0.300663,0.684342,0.389382,0.242658,0.584519,0.137125,0.380047,0.593610,0.107091,...,0.367867,0.249902,0.320618,0.642996,0.387693,0.123840,0.304177,0.430744,0.291792,0.470880
2,9b314ad7-fbcb0422-6db62dfc-732858d0-a5527d8b,0.221213,0.816509,0.584521,0.295778,0.759532,0.077791,0.699801,0.809538,0.094591,...,0.458704,0.385597,0.543361,0.516679,0.498094,0.094852,0.372938,0.291618,0.184312,0.701100
3,6a5c3985-7764bdd0-ec5a6a74-af78bcaa-4ca33ec3,0.423000,0.526573,0.382708,0.436789,0.498957,0.305980,0.294822,0.381068,0.383320,...,0.547701,0.412795,0.410245,0.458032,0.503042,0.313026,0.509457,0.394524,0.275012,0.409420
4,c50494f1-90e2bff5-e9189550-1a4562fd-6ab5204c,0.443502,0.458464,0.369382,0.481704,0.462845,0.315876,0.308608,0.397049,0.460156,...,0.619278,0.448103,0.395425,0.638857,0.500293,0.369428,0.631260,0.349194,0.341160,0.507027


In [38]:
df_predictions.to_csv('submission.csv', index=False)